In [7]:
import os
import sys
import typing

from collections import namedtuple

import numpy as np
import pandas as pd

from nvdlib.nvd import NVD

In [8]:
sys.path.append('../src/')

In [9]:
from f8a_version_comparator.comparable_version import ComparableVersion

from toolkit.preprocessing import NVDFeedPreprocessor

In [10]:
feed = NVD.from_recent()
# feed.update()

data = list(feed.cves())

In [11]:
transformed = NVDFeedPreprocessor(attributes=['cve_id', 'configurations']).fit_transform(data, use_filter=False)

In [12]:
# Series.configurations contains List[ConfigurationNode]
transformed_filtered = list(filter(lambda e: any(e.configurations), transformed))

In [13]:
df = pd.DataFrame(transformed_filtered).drop('repository', axis=1)
df

,user,project,cve_id,configurations
0,None,None,CVE-2014-0900,[<nvdlib.model.ConfigurationNode object at 0x7...
1,None,None,CVE-2014-0950,[<nvdlib.model.ConfigurationNode object at 0x7...
2,None,None,CVE-2014-5014,[<nvdlib.model.ConfigurationNode object at 0x7...
3,None,None,CVE-2016-10258,[<nvdlib.model.ConfigurationNode object at 0x7...
4,cloudfoundry,cloud_controller_ng,CVE-2016-2169,[<nvdlib.model.ConfigurationNode object at 0x7...
5,None,None,CVE-2016-8220,[<nvdlib.model.ConfigurationNode object at 0x7...
6,None,None,CVE-2016-9092,[<nvdlib.model.ConfigurationNode object at 0x7...
7,None,None,CVE-2016-9093,[<nvdlib.model.ConfigurationNode object at 0x7...
8,curl,curl,CVE-2016-9586,[<nvdlib.model.ConfigurationNode object at 0x7...
9,None,None,CVE-2016-9592,[<nvdlib.model.ConfigurationNode object at 0x7...


In [14]:
def get_index(obj, index):
    try:
        el = obj[index]
    except IndexError:
        el = None
        pass
    
    return el

# cpe_base_list = list(
#     filter(
#         lambda e: e is not None,
#         [get_index(transformed_filtered[i].configurations[0].cpe, 0)
#         for i in range(len(transformed_filtered))]
#     )
# )

In [23]:
from itertools import compress

CPE_VERSION_ATTRIBUTE_LIST = [
    'vendor',
    'product',
    'versionExact',
    'versionStartExcluding',
    'versionStartIncluding',
    'versionEndIncluding',
    'versionEndExcluding',
]


VICTIM_VERSION_OPERATOR_LIST = ['==', '>', '>=', '<=', '<']


class VersionNode(namedtuple('VersionNode', CPE_VERSION_ATTRIBUTE_LIST)):
    
    def __new__(cls, cpe=None):
        return super(VersionNode, cls).__new__(
            cls,
            **{attr: getattr(cpe, attr, None) for attr in CPE_VERSION_ATTRIBUTE_LIST}
        )
    
    def __lt__(self, other):
        raise NotImplementedError
    
    def __gt__(self, other):
        raise NotImplementedError
    
    def __eq__(self, other):
        if (self.vendor, self.product) != (other.vendor, other.product):
            return False
        
        return all([
            ComparableVersion(ver_a).compare_to(ComparableVersion(ver_b))
            for ver_a, ver_b in zip(self[2:], other[2:])
        ])
        
    @classmethod
    def from_cpe(cls, cpe_list: list) -> list:
        if not cpe_list:
            return None
        
        return [cls(cpe) for cpe in cpe_list]
    
    @property
    def victims_notation(self):
        return VersionNode.get_victims_notation(self)
    
    @staticmethod
    def get_victims_notation(other: typing.Union["VersionNode", tuple]):
        version_string =  ",".join([
            f"{op}{version}" for op, version in zip(VICTIM_VERSION_OPERATOR_LIST, other)
            if version is not None
        ])
        
        return version_string

In [25]:
version_nodes = list()
for config in df.configurations:
    for node in config:
        version_ranges = VersionNode.from_cpe(node.cpe)
        version_nodes.extend(version_ranges or [])

In [26]:
version_series = pd.Series(version_nodes)
version_series.dropna(inplace=True)
version_series.drop_duplicates(inplace=True)

pd.DataFrame.from_records(version_series.values.tolist(), columns=CPE_VERSION_ATTRIBUTE_LIST)

,vendor,product,versionExact,versionStartExcluding,versionStartIncluding,versionEndIncluding,versionEndExcluding
0,google,android,None,None,None,4.4.1,None
1,ibm,rational_clearquest,None,None,7.1.1,7.1.1.9,None
2,ibm,rational_clearquest,None,None,7.1.2,7.1.2.13,None
3,ibm,rational_clearquest,None,None,8.0.0,8.0.0.10,None
4,ibm,rational_clearquest,None,None,8.0.1,8.0.1.3,None
5,tinywebgallery,wordpress_flash_uploader,None,None,None,None,3.1.3
6,symantec,advanced_secure_gateway,None,None,6.6,None,6.6.5.14
7,symantec,advanced_secure_gateway,None,None,6.7,None,6.7.3.1
8,symantec,proxysg,None,None,6.5,None,6.5.10.8
9,symantec,proxysg,None,None,6.6,None,6.6.5.14


In [43]:
a = version_series[16]
b = version_series[17]

In [39]:
from f8a_version_comparator import *

In [56]:
a = ComparableVersion('1.0.0')
b = ComparableVersion('1.0.0')

In [30]:
a == b

AttributeError: 'ComparableVersion' object has no attribute 'items'